# LTIM time series, 1991 to 2016 Age and Sex

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'internationalmigration/datasets/longterminternationalmigrationageandsextable207')
scraper

## Long-term international migration 2.07, age and sex, UK and England and Wales

Age and sex of migrants entering or leaving UK. Estimates of Long-Term International Migration, annual table.

### Distributions

1. Long-term international migration 2.07, age and sex, UK and England and Wales ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/longterminternationalmigrationageandsextable207/current/2.07ltimageandsex1991to2017.xls))


In [2]:
tab = next(t for t in scraper.distribution(latest=True, mediaType=Excel).as_databaker() if t.name == 'Table 2.07')

In [3]:
corner = tab.filter('Year')
corner.assert_one()

observations = corner \
    .shift(RIGHT) \
    .fill(DOWN) \
    .filter('Estimate') \
    .expand(RIGHT) \
    .filter('Estimate') \
    .fill(DOWN) \
    .is_not_blank() \
    .is_not_whitespace() \
    .filter(lambda x: type(x.value) != str or 'Statistically Significant Decrease' not in x.value)
observations = observations - (tab.excel_ref('A1')
                               .expand(DOWN)
                               .filter(contains_string('Significant Change'))
                               .expand(RIGHT)
                              )
original_estimates = tab \
    .filter(contains_string('Original Estimates')) \
    .fill(DOWN) \
    .is_number()

observations = observations - original_estimates
savepreviewhtml([observations, original_estimates])

In [4]:
CI = observations.shift(RIGHT)
Year = corner.fill(DOWN) & \
    observations.fill(LEFT)
Geography = corner.fill(DOWN).one_of(['United Kingdom', 'England and Wales'])
Age = corner.fill(RIGHT).is_not_blank()
Age_dim = HDim(Age, 'Age', CLOSEST, LEFT)
Age_dim.AddCellValueOverride('45-59/642', '45-59/64')
Age_dim.AddCellValueOverride('60/65 and over3', '60/65 and over')
Age_dim.AddCellValueOverride('All Ages', 'All ages')
Sex = corner.shift(DOWN).fill(RIGHT).is_not_blank()
Flow = corner.fill(DOWN).one_of(['Inflow', 'Outflow', 'Balance'])
csObs = ConversionSegment(observations, [
    HDim(Year,'Year', DIRECTLY, LEFT),
    HDim(Geography,'Geography', CLOSEST, ABOVE),
    Age_dim,
    HDim(Sex, 'Sex', CLOSEST, LEFT),
    HDim(Flow, 'Migration Flow', CLOSEST, ABOVE),
    HDimConst('Measure Type', 'Count'),
    HDimConst('Unit','People (thousands)'),
    HDim(CI,'CI',DIRECTLY,RIGHT),
])
savepreviewhtml(csObs)

In [5]:
tidy_revised = csObs.topandas()
tidy_revised

,OBS,Year,Geography,Age,Sex,Migration Flow,Measure Type,Unit,CI
0,329.0,1991.0,United Kingdom,All ages,Persons,Inflow,Count,People (thousands),23.0
1,157.0,1991.0,United Kingdom,All ages,Males,Inflow,Count,People (thousands),15.0
2,172.0,1991.0,United Kingdom,All ages,Females,Inflow,Count,People (thousands),18.0
3,53.0,1991.0,United Kingdom,Under 15,Persons,Inflow,Count,People (thousands),10.0
4,23.0,1991.0,United Kingdom,Under 15,Males,Inflow,Count,People (thousands),6.0
5,30.0,1991.0,United Kingdom,Under 15,Females,Inflow,Count,People (thousands),8.0
6,107.0,1991.0,United Kingdom,15-24,Persons,Inflow,Count,People (thousands),12.0
7,47.0,1991.0,United Kingdom,15-24,Males,Inflow,Count,People (thousands),8.0
8,59.0,1991.0,United Kingdom,15-24,Females,Inflow,Count,People (thousands),9.0
9,139.0,1991.0,United Kingdom,25-44,Persons,Inflow,Count,People (thousands),16.0


Also need to pull out the group of original estimates

In [6]:
csRevs = ConversionSegment(original_estimates, [
    HDim(Year, 'Year', DIRECTLY, LEFT),
    HDim(Geography,'Geography', CLOSEST, ABOVE),
    Age_dim,
    HDim(Sex, 'Sex', CLOSEST, LEFT),
    HDim(Flow, 'Migration Flow', CLOSEST, ABOVE),
    HDimConst('Measure Type', 'Count'),
    HDimConst('Unit','People (thousands)'),
    HDim(original_estimates.shift(RIGHT), 'CI', DIRECTLY, RIGHT),
    HDimConst('Revision', 'Original Estimate')
])
savepreviewhtml(csRevs)

In [7]:
orig_estimates = csRevs.topandas()
orig_estimates

,OBS,Year,Geography,Age,Sex,Migration Flow,Measure Type,Unit,CI,Revision
0,171.0,2001.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),40.0,Original Estimate
1,153.0,2002.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),43.0,Original Estimate
2,148.0,2003.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),46.0,Original Estimate
3,245.0,2004.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),49.0,Original Estimate
4,206.0,2005.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),49.0,Original Estimate
5,198.0,2006.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),52.0,Original Estimate
6,233.0,2007.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),48.0,Original Estimate
7,163.0,2008.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),57.0,Original Estimate
8,198.0,2009.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),37.0,Original Estimate
9,252.0,2010.0,United Kingdom,All ages,Persons,Balance,Count,People (thousands),37.0,Original Estimate


In [8]:
tidy = pd.concat([tidy_revised, orig_estimates], axis=0, join='outer', ignore_index=True, sort=False)
tidy

,OBS,Year,Geography,Age,Sex,Migration Flow,Measure Type,Unit,CI,Revision
0,329.0,1991.0,United Kingdom,All ages,Persons,Inflow,Count,People (thousands),23.0,NaN
1,157.0,1991.0,United Kingdom,All ages,Males,Inflow,Count,People (thousands),15.0,NaN
2,172.0,1991.0,United Kingdom,All ages,Females,Inflow,Count,People (thousands),18.0,NaN
3,53.0,1991.0,United Kingdom,Under 15,Persons,Inflow,Count,People (thousands),10.0,NaN
4,23.0,1991.0,United Kingdom,Under 15,Males,Inflow,Count,People (thousands),6.0,NaN
5,30.0,1991.0,United Kingdom,Under 15,Females,Inflow,Count,People (thousands),8.0,NaN
6,107.0,1991.0,United Kingdom,15-24,Persons,Inflow,Count,People (thousands),12.0,NaN
7,47.0,1991.0,United Kingdom,15-24,Males,Inflow,Count,People (thousands),8.0,NaN
8,59.0,1991.0,United Kingdom,15-24,Females,Inflow,Count,People (thousands),9.0,NaN
9,139.0,1991.0,United Kingdom,25-44,Persons,Inflow,Count,People (thousands),16.0,NaN


In [9]:
original_slice = tidy[tidy['Revision'] == 'Original Estimate']
tidy['Revision'] = tidy.apply(
    lambda row: '2011 Census Revision' if row['CI'] == ':' else 'Original Estimate',
    axis=1
)
tidy

,OBS,Year,Geography,Age,Sex,Migration Flow,Measure Type,Unit,CI,Revision
0,329.0,1991.0,United Kingdom,All ages,Persons,Inflow,Count,People (thousands),23.0,Original Estimate
1,157.0,1991.0,United Kingdom,All ages,Males,Inflow,Count,People (thousands),15.0,Original Estimate
2,172.0,1991.0,United Kingdom,All ages,Females,Inflow,Count,People (thousands),18.0,Original Estimate
3,53.0,1991.0,United Kingdom,Under 15,Persons,Inflow,Count,People (thousands),10.0,Original Estimate
4,23.0,1991.0,United Kingdom,Under 15,Males,Inflow,Count,People (thousands),6.0,Original Estimate
5,30.0,1991.0,United Kingdom,Under 15,Females,Inflow,Count,People (thousands),8.0,Original Estimate
6,107.0,1991.0,United Kingdom,15-24,Persons,Inflow,Count,People (thousands),12.0,Original Estimate
7,47.0,1991.0,United Kingdom,15-24,Males,Inflow,Count,People (thousands),8.0,Original Estimate
8,59.0,1991.0,United Kingdom,15-24,Females,Inflow,Count,People (thousands),9.0,Original Estimate
9,139.0,1991.0,United Kingdom,25-44,Persons,Inflow,Count,People (thousands),16.0,Original Estimate


Check each observation has a year and use ints.

In [10]:
tidy['Year'] = tidy['Year'].apply(lambda x: pd.to_numeric(x, downcast='integer'))

Ignore data markers for now and ensure all observations and confidence intervals are integers.

**Todo: figure out what to do with data markers.**

In [11]:
import numpy as np
tidy['OBS'].replace('', np.nan, inplace=True)
tidy.dropna(subset=['OBS'], inplace=True)
if 'DATAMARKER' in tidy.columns:
    tidy.drop(columns=['DATAMARKER'], inplace=True)
tidy.rename(columns={'OBS': 'Value'}, inplace=True)
tidy['Value'] = tidy['Value'].astype(int)
tidy['CI'] = tidy['CI'].map(lambda x:
                            '' if x == ':' else int(x[:-2]) if x.endswith('.0') else 'ERR')
tidy

,Value,Year,Geography,Age,Sex,Migration Flow,Measure Type,Unit,CI,Revision
0,329,1991,United Kingdom,All ages,Persons,Inflow,Count,People (thousands),23,Original Estimate
1,157,1991,United Kingdom,All ages,Males,Inflow,Count,People (thousands),15,Original Estimate
2,172,1991,United Kingdom,All ages,Females,Inflow,Count,People (thousands),18,Original Estimate
3,53,1991,United Kingdom,Under 15,Persons,Inflow,Count,People (thousands),10,Original Estimate
4,23,1991,United Kingdom,Under 15,Males,Inflow,Count,People (thousands),6,Original Estimate
5,30,1991,United Kingdom,Under 15,Females,Inflow,Count,People (thousands),8,Original Estimate
6,107,1991,United Kingdom,15-24,Persons,Inflow,Count,People (thousands),12,Original Estimate
7,47,1991,United Kingdom,15-24,Males,Inflow,Count,People (thousands),8,Original Estimate
8,59,1991,United Kingdom,15-24,Females,Inflow,Count,People (thousands),9,Original Estimate
9,139,1991,United Kingdom,25-44,Persons,Inflow,Count,People (thousands),16,Original Estimate


In [12]:
for col in tidy.columns:
    if col not in ['Value', 'Year', 'CI']:
        tidy[col] = tidy[col].astype('category')
        display(col)
        display(tidy[col].cat.categories)

'Geography'

Index(['England and Wales', 'United Kingdom'], dtype='object')

'Age'

Index(['15-24', '25-44', '45-59/64', '60/65 and over', 'All ages', 'Under 15'], dtype='object')

'Sex'

...

Index(['Count'], dtype='object')

'Unit'

Index(['People (thousands)'], dtype='object')

'Revision'

Index(['2011 Census Revision', 'Original Estimate'], dtype='object')

In [13]:
tidy['Geography'] = tidy['Geography'].cat.rename_categories({
    'United Kingdom': 'K02000001',
    'England and Wales': 'K04000001'
})
tidy['Age'] = tidy['Age'].cat.rename_categories({
    '15-24': 'agr/15-24',
    '25-44': 'agr/25-44',
    '45-59/64': 'agr/45-59-or-64',
    '60/65 and over': 'agr/60-or-65-and-over',
    'All ages': 'all',
    'Under 15': 'agr/under-15'
})
tidy['Sex'] = tidy['Sex'].cat.rename_categories({
    'Females': 'F',
    'Males': 'M',
    'Persons': 'T'
})
tidy['Migration Flow'].cat.categories = tidy['Migration Flow'].cat.categories.map(pathify)

tidy = tidy[['Geography', 'Year', 'Age', 'Sex', 'Migration Flow',
             'Value', 'Measure Type', 'Unit', 'CI', 'Revision']]
tidy

,Geography,Year,Age,Sex,Migration Flow,Value,Measure Type,Unit,CI,Revision
0,K02000001,1991,all,T,inflow,329,Count,People (thousands),23,Original Estimate
1,K02000001,1991,all,M,inflow,157,Count,People (thousands),15,Original Estimate
2,K02000001,1991,all,F,inflow,172,Count,People (thousands),18,Original Estimate
3,K02000001,1991,agr/under-15,T,inflow,53,Count,People (thousands),10,Original Estimate
4,K02000001,1991,agr/under-15,M,inflow,23,Count,People (thousands),6,Original Estimate
5,K02000001,1991,agr/under-15,F,inflow,30,Count,People (thousands),8,Original Estimate
6,K02000001,1991,agr/15-24,T,inflow,107,Count,People (thousands),12,Original Estimate
7,K02000001,1991,agr/15-24,M,inflow,47,Count,People (thousands),8,Original Estimate
8,K02000001,1991,agr/15-24,F,inflow,59,Count,People (thousands),9,Original Estimate
9,K02000001,1991,agr/25-44,T,inflow,139,Count,People (thousands),16,Original Estimate


In [14]:
from pathlib import Path
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

tidy.to_csv(destinationFolder / ('observations.csv'), index = False)

In [15]:
from gssutils.metadata import THEME

scraper.dataset.family = 'migration'
scraper.dataset.theme = THEME['population']
scraper.dataset.license = 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'

with open(destinationFolder / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())